# Scenario Ingestion Notebook

This notebook ingests scenario markdown files into ChromaDB for RAG retrieval.

**Purpose:** Read all 18 scenario files from `/scenarios` and embed them using Gemini Embedding 001.

**Run this once before starting the app.**

## Setup

Import required libraries and configure paths.

In [1]:
import os
import sys
from pathlib import Path
import re

# Add parent directory to path to import backend modules
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

from backend.config import get_config

print("✅ Imports successful")

/Users/i583134/.pyenv/versions/3.13.2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports successful


## Configuration

Load configuration and validate API key.

In [2]:
# Get configuration
config = get_config()

try:
    config.validate()
    print("✅ Configuration validated")
    print(f"📊 Embedding model: {config.embedding_model}")
    print(f"💾 ChromaDB path: {config.chroma_db_path}")
    print(f"📦 Collection name: {config.chroma_collection_name}")
except ValueError as e:
    print(f"❌ Configuration error: {e}")
    print("\nMake sure you have:")
    print("1. Created .env file in app/ directory")
    print("2. Added your GOOGLE_API_KEY to .env")

✅ Configuration validated
📊 Embedding model: models/gemini-embedding-001
💾 ChromaDB path: ./app/data/chroma_db
📦 Collection name: ot_scenarios


## Helper Functions

In [3]:
def extract_scenario_title(content: str) -> str:
    """
    Extract scenario title from markdown heading.
    
    Args:
        content: Markdown file content
    
    Returns:
        Scenario title, or "Untitled" if not found
    """
    # Look for first heading (# Scenario XX: Title)
    match = re.search(r'^#\s+Scenario\s+\d+:\s*(.+)$', content, re.MULTILINE)
    if match:
        return match.group(1).strip()
    
    # Fallback: look for any first-level heading
    match = re.search(r'^#\s+(.+)$', content, re.MULTILINE)
    if match:
        return match.group(1).strip()
    
    return "Untitled Scenario"

print("✅ Helper functions defined")

✅ Helper functions defined


## Initialize Embedding Model

Create Gemini Embedding 001 model instance.

In [4]:
print("📊 Initializing embedding model...")

embeddings = GoogleGenerativeAIEmbeddings(
    model=config.embedding_model,
    google_api_key=config.google_api_key
)

print("✅ Embedding model initialized")

📊 Initializing embedding model...
✅ Embedding model initialized


## Initialize ChromaDB

Create or connect to ChromaDB vector store.

In [5]:
print(f"💾 Initializing ChromaDB at: {config.chroma_db_path}")

vector_store = Chroma(
    collection_name=config.chroma_collection_name,
    embedding_function=embeddings,
    persist_directory=config.chroma_db_path
)

print("✅ ChromaDB initialized")

# Check if collection already has documents
try:
    existing_count = vector_store._collection.count()
    if existing_count > 0:
        print(f"⚠️  Collection already contains {existing_count} documents")
        print("   Running this notebook will ADD more documents (duplicates possible)")
        print("   Consider deleting the chroma_db folder first if you want a clean start")
except:
    print("📝 Collection is empty (first-time ingestion)")

💾 Initializing ChromaDB at: ./app/data/chroma_db
✅ ChromaDB initialized


## Load Scenario Files

Find and read all scenario markdown files.

In [6]:
# Find scenarios directory (go up two levels from app/scripts to project root)
scenarios_dir = Path.cwd().parent.parent / "scenarios"

if not scenarios_dir.exists():
    raise FileNotFoundError(f"❌ Scenarios directory not found: {scenarios_dir}")

scenario_files = sorted(scenarios_dir.glob("scenario-*.md"))

if not scenario_files:
    raise FileNotFoundError(f"❌ No scenario files found in: {scenarios_dir}")

print(f"📁 Found {len(scenario_files)} scenario files:")
for f in scenario_files:
    print(f"   - {f.name}")

📁 Found 18 scenario files:
   - scenario-1.md
   - scenario-10.md
   - scenario-11.md
   - scenario-12.md
   - scenario-13.md
   - scenario-14.md
   - scenario-15.md
   - scenario-16.md
   - scenario-17.md
   - scenario-18.md
   - scenario-2.md
   - scenario-3.md
   - scenario-4.md
   - scenario-5.md
   - scenario-6.md
   - scenario-7.md
   - scenario-8.md
   - scenario-9.md


## Process Scenarios

Read each scenario file and create Document objects.

In [7]:
documents = []
errors = []

print("\n🔄 Processing scenario files...\n")

for i, file_path in enumerate(scenario_files, 1):
    print(f"[{i}/{len(scenario_files)}] {file_path.name}")
    
    try:
        # Read file
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extract metadata
        scenario_id = file_path.stem  # e.g., "scenario-01"
        title = extract_scenario_title(content)
        
        print(f"  ├─ ID: {scenario_id}")
        print(f"  ├─ Title: {title}")
        print(f"  └─ Length: {len(content)} chars\n")
        
        # Create document
        doc = Document(
            page_content=content,
            metadata={
                "id": scenario_id,
                "title": title,
                "source": file_path.name
            }
        )
        
        documents.append(doc)
        
    except Exception as e:
        print(f"  ❌ Error: {str(e)}\n")
        errors.append((file_path.name, str(e)))

print(f"\n✅ Successfully processed {len(documents)}/{len(scenario_files)} scenarios")

if errors:
    print(f"\n⚠️  Errors encountered:")
    for filename, error in errors:
        print(f"   - {filename}: {error}")


🔄 Processing scenario files...

[1/18] scenario-1.md
  ├─ ID: scenario-1
  ├─ Title: Refusal of Treatment by Student
  └─ Length: 6137 chars

[2/18] scenario-10.md
  ├─ ID: scenario-10
  ├─ Title: Coping with Resistance (Spinal Cord Injury)
  └─ Length: 6187 chars

[3/18] scenario-11.md
  ├─ ID: scenario-11
  ├─ Title: Interprofessional Conflict in Psych Ward
  └─ Length: 5975 chars

[4/18] scenario-12.md
  ├─ ID: scenario-12
  ├─ Title: Ethical Dilemma & Involuntary Hospitalization
  └─ Length: 6781 chars

[5/18] scenario-13.md
  ├─ ID: scenario-13
  ├─ Title: Financial Ethics & Divorce in Mental Health
  └─ Length: 5921 chars

[6/18] scenario-14.md
  ├─ ID: scenario-14
  ├─ Title: Cultural Competence and Parental Engagement
  └─ Length: 7444 chars

[7/18] scenario-15.md
  ├─ ID: scenario-15
  ├─ Title: Language Barrier & Cultural Competence (Russian)
  └─ Length: 5289 chars

[8/18] scenario-16.md
  ├─ ID: scenario-16
  ├─ Title: Conflict with Customer over Equipment
  └─ Length: 388

## Embed and Store

Generate embeddings and store in ChromaDB.

**Note:** This step may take a few minutes as it calls the Google AI API for each scenario.

In [8]:
if not documents:
    raise ValueError("❌ No documents to ingest")

print(f"🔄 Embedding and storing {len(documents)} documents...")
print("⏳ This may take a few minutes...\n")

try:
    ids = vector_store.add_documents(documents=documents)
    print(f"\n✅ Successfully ingested {len(ids)} scenarios")
    
    # Verify
    collection = vector_store._collection
    count = collection.count()
    print(f"📊 ChromaDB collection '{config.chroma_collection_name}' now contains {count} documents")
    
except Exception as e:
    print(f"\n❌ Ingestion failed: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

🔄 Embedding and storing 18 documents...
⏳ This may take a few minutes...


✅ Successfully ingested 18 scenarios
📊 ChromaDB collection 'ot_scenarios' now contains 18 documents


## Verification

Test a sample query to verify the embeddings work.

In [13]:
print("🧪 Testing retrieval with sample query...\n")

test_query = '''
אני מרפאה בעיסוק מתחילה, סיימתי את לימודיי לפני כחצי שנה והיום אני פוגשת את המטופל הראשון שלי. המטופל שלי הוא בן 56, הוא גבר עצמאי עם אגו גברי. הוא עובד בכיר בעירייה שמכיר את כולם וכולם מכירים אותו. הוא רגיל להיות עצמאי תמיד וחשוב לו נורא הכבוד העצמי שלו. הוא עבר תאונת דרכים קשה ובעקבות זה הוא מתנייד בכיסא גלגלים. הוא נפצע קשה ברגליו והוא צריך ללמוד מחדש איך ללכת. תייעץ לי בבקשה על רעיונות של איך להתחיל את המפגש הראשון שלנו.
'''
results = vector_store.similarity_search_with_score(test_query, k=2)

print(f"Query: '{test_query}'\n")
print("Top 2 matches:")
for i, (doc, score) in enumerate(results, 1):
    print(f"\n{i}. {doc.metadata['title']}")
    print(f"   ID: {doc.metadata['id']}")
    print(f"   Score: {score:.4f}")
    print(f"   Preview: {doc.page_content[:100]}...")

print("\n✅ Retrieval test successful!")

🧪 Testing retrieval with sample query...

Query: '
אני מרפאה בעיסוק מתחילה, סיימתי את לימודיי לפני כחצי שנה והיום אני פוגשת את המטופל הראשון שלי. המטופל שלי הוא בן 56, הוא גבר עצמאי עם אגו גברי. הוא עובד בכיר בעירייה שמכיר את כולם וכולם מכירים אותו. הוא רגיל להיות עצמאי תמיד וחשוב לו נורא הכבוד העצמי שלו. הוא עבר תאונת דרכים קשה ובעקבות זה הוא מתנייד בכיסא גלגלים. הוא נפצע קשה ברגליו והוא צריך ללמוד מחדש איך ללכת. תייעץ לי בבקשה על רעיונות של איך להתחיל את המפגש הראשון שלנו.
'

Top 2 matches:

1. Refusal of Treatment by Student
   ID: scenario-1
   Score: 0.4451
   Preview: # Scenario 01: Refusal of Treatment by Student
**Topic:** Professional Identity & Patient Resistance...

2. Coping with Resistance (Spinal Cord Injury)
   ID: scenario-10
   Score: 0.4894
   Preview: # Scenario 10: Coping with Resistance (Spinal Cord Injury)
**Topic:** Rehabilitation & Emotional Res...

✅ Retrieval test successful!


## Summary

Ingestion complete! The ChromaDB database is now ready for use by the Streamlit app.

In [14]:
print("\n" + "="*60)
print("✨ INGESTION COMPLETE ✨")
print("="*60)
print(f"\n📊 Total scenarios ingested: {len(documents)}")
print(f"💾 Database location: {config.chroma_db_path}")
print(f"📦 Collection name: {config.chroma_collection_name}")
print("\n🚀 You can now run the Streamlit app with:")
print("   cd ..")
print("   streamlit run app.py")
print("\n" + "="*60)


✨ INGESTION COMPLETE ✨

📊 Total scenarios ingested: 18
💾 Database location: ./app/data/chroma_db
📦 Collection name: ot_scenarios

🚀 You can now run the Streamlit app with:
   cd ..
   streamlit run app.py



In [15]:
import pandas as pd

# Get all documents from collection
collection = vector_store._collection
all_docs = collection.get(include=['metadatas', 'documents'])

# Create DataFrame for easy viewing
df = pd.DataFrame({
    'ID': [m['id'] for m in all_docs['metadatas']],
    'Title': [m['title'] for m in all_docs['metadatas']],
    'Source': [m['source'] for m in all_docs['metadatas']],
    'Content Length': [len(d) for d in all_docs['documents']]
})

print(f"📊 ChromaDB contains {len(df)} documents:\n")
print(df.to_string(index=False))

# Optional: Display full content of a specific scenario
print("\n" + "="*60)
print("💡 To view full content of a specific scenario, run:")
print("   doc_index = 0  # Change to desired row number")
print("   print(all_docs['documents'][doc_index])")
print("="*60)

📊 ChromaDB contains 18 documents:

         ID                                            Title         Source  Content Length
 scenario-1                  Refusal of Treatment by Student  scenario-1.md            6137
scenario-10      Coping with Resistance (Spinal Cord Injury) scenario-10.md            6187
scenario-11         Interprofessional Conflict in Psych Ward scenario-11.md            5975
scenario-12    Ethical Dilemma & Involuntary Hospitalization scenario-12.md            6781
scenario-13      Financial Ethics & Divorce in Mental Health scenario-13.md            5921
scenario-14      Cultural Competence and Parental Engagement scenario-14.md            7444
scenario-15 Language Barrier & Cultural Competence (Russian) scenario-15.md            5289
scenario-16            Conflict with Customer over Equipment scenario-16.md            3883
scenario-17        Stroke Patient and Husband's Interference scenario-17.md            3088
scenario-18        Stroke Patient and Husband

## Inspect ChromaDB Contents

View all documents stored in the database as a table.